In [2]:
import os
import pandas as pd
import numpy as np
from scipy import stats
import plotly.io as pio
from plotly.subplots import make_subplots 
import importlib
from jw_utils import rna_seq_utils as rsu
from jw_utils import plotly_preferences as pprefs
from jw_utils import parse_gff as pgf
from pathlib import Path

In [63]:
FEATURE_CNTS_DIR = Path('./results/featureCounts/')
PCA_DIR = Path('./results/PCA_analysis')
PCA_DIR.mkdir(exist_ok=True, parents=True)
SAMPLESHEET_FP = './assets/samplesheet.csv'

In [107]:
conditions = ['wt_1_light_T1', 'wt_2_light_T1', 'wt_3_light_T1',
              'wt_1_dark_T1', 'wt_2_dark_T1', 'wt_3_dark_T1',
              'dimA_1_dark_T1', 'dimA_2_dark_T1', 'dimA_3_dark_T1',
              'dimA_1_light_T1', 'dimA_2_light_T1','dimA_3_light_T1',
             ]
dfs = []
for c in conditions:
    fp = FEATURE_CNTS_DIR / f'{c}.txt'
    df = pd.read_csv(fp, sep='\t', skiprows=1, index_col=0)
    name=c.replace('_T1', '')
    df = df.rename(columns={df.columns[-1]:name})[[name]]
    dfs.append(df)
    
df = pd.concat(dfs, axis=1, join='outer')
df.to_csv(PCA_DIR / 'count_data.csv')

def rpm(ser):
    return (ser / ser.sum())*1000000

df_rpm = df
for col in df_rpm:
    df_rpm[col] = rpm(df_rpm[col])

df_rpm.to_csv(PCA_DIR / 'count_data_rpm.csv')

In [90]:
samplesheet_df = pd.read_csv(SAMPLESHEET_FP)
samp_info_df = samplesheet_df[['sample', 'condition']]
samp_info_df = samp_info_df[(samp_info_df['condition'] != 'bphP_light')]
samp_info_df = samp_info_df.set_index('sample')
samp_info_df.to_csv(PCA_DIR / 'col_data.csv')